In [2]:
# Implementation of the Eisenberg and Noe 2001 Debt Model in Python
# Eisenberg and Noe 2001 analyse the properties of intercorporate cash flows in financial 
# systems featuring cyclical interdependence and endogenously determined clearing vectors.
# The model computes clearing vectors for interlinked financial systems. A clearing vector is 
# a vector of payments from nodes in the financial system to other nodes and must satisfy the 
# conditions of proportional repayments of liabilities in default (we assume all debt claims have equal priority), limited liability of equity
# and absolute priority of debt over equity. The clearing vector is computed through a "fictitious 
# sequential default" algorithm in which the set of defaulting firms at the start of each round is fixed by
# the dynamic adjustments of the system from the preceding round. In each new round,an attempt to clear the 
# system that assumes that only nodes that defaulted in the previous round default. If no new defaults occur, 
# the algorithm terminates (i.e. a node is a distincy economig entity or a financial node i.e. a firm) 

# This algorithm gives the clearing vector and a natural measure of systemic risk (i.e. the exposure of a given node
# in the system to defaults by other firms - this is based on the number of waves of defaults required to 
# induce a given firm in the system to fail)

# Julian Kanjere, knjjul001@myuct.ac.za, September 2021
######### IMPORTS ######### 

import numpy as np

######### /IMPORTS ######### 

######### MODEL SETUP ######### 

NETWORK_TYPE = 'STAR' #'CIRCULANT'
NUM_AGENTS = 5 # i.e. n

# this matrix captures the nominal liability of one node to another in the system as a proportion of the debtor node's total liabilites
LIABILITY_MATRIX = np.zeros((NUM_AGENTS, NUM_AGENTS)) 

# n x n liabilities matrix where Lij is debt that agent i owes agent j
RELATIVE_LIABILITY_MATRIX = np.zeros((NUM_AGENTS, NUM_AGENTS)) 

OPERATING_CASH_FLOW_VECTOR = {} # a dictionary whose key is the round starting from 1 and value is a list of exogenous cash infusion to a node i.e. from outside sources

TOTAL_DOLLAR_PAYMENT_VECTOR = {} # a dictionary whose key is the round starting from 1 and value is a list of total payments made by each node to other nodes in the system i.e. p = (p1, p2, p3, ...)

TOTAL_OBLIGATION_VECTOR = {} # a dictionary whose key is the round starting from 1 and value is a list of payments made by each node to other nodes in the system i.e. p = (p1, p2, p3, ...)

CLEARING_PAYMENT_VECTOR = {} # a dictionary whose key is the round starting from 1 and value is a list of payments

######### /MODEL SETUP ######### 

######### HELPER FUNCTIONS ######### 

def bool_limited_liability():
    pass

def bool_proportional_repay():
    pass

def bool_debt_over_equity():
    pass


def bool_check_defaults(round):
    '''Function to check whether there are any defaults in a given round. If not, algorithm can terminate'''
    pass

def calculate_operating_cash_flow_for_node(i):
    '''Function to calculate operating cash flow to node i in a given round which is the exogenous operating
    cash flow received by node i'''
    pass

def calculate_total_cash_flow_for_node(i, total_cash_flow, liabilities_received):
    '''Function to calculate total cash flow to node i in a given round which is liabilities received (endogenous) plus
    operating cashflow (exogenous)
    '''
    pass

def calculate_liabilities_out_for_node(i):
    '''Function to calculate liabilities out for node i in a given round. These nominal liabilities represent the 
    promised payments due to other nodes in the system. This is represented in the nominal liabilities matrix
    LIABILITY_MATRIX
    '''
    pass

def calculate_total_equity_for_node(i, total_cash_flow, liabilities_out):
    '''function to calculate total equity of node i in a given round which is total cash flow (i.e. liabilities received 
    (endogenous) plus operating cashflow (exogenous)) minus liabilities out (i.e. nominal payments)
    '''
    return total_cash_flow - liabilities_out

def calculate_total_value_for_node(i, equity, liabilities_out):
    '''Function to calculate total value of node i in a given round which is debt plus equity'''
    return total_cash_flow - liabilities_out

def update_clearing_payment_vector(r):
    '''Function to update the clearing payment vector for a given round r'''
    pass

def calculate_systemic_risk(r):
    '''Function to calculate systemic risk for each firm in a given round i.e. this is based on the number of 
    waves of defaults required to induce a given firm in the system to fail'''
    pass

######### /HELPER FUNCTIONS ######### 

